In [1]:
import pandas as pd 
import sqlite3 

In [2]:
df = pd.read_csv("Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv", encoding='ISO-8859-1')
df.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5910,Export Quantity,2016,2016,tonnes,3.0,*
1,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5922,Export Value,2016,2016,1000 US$,23.0,*
2,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2015,2015,1000 US$,1.0,*
3,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2016,2016,1000 US$,1.0,*
4,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2017,2017,1000 US$,5.0,R


In [3]:
df.dtypes 

Reporter Country Code      int64
Reporter Countries        object
Partner Country Code       int64
Partner Countries         object
Item Code                  int64
Item                      object
Element Code               int64
Element                   object
Year Code                  int64
Year                       int64
Unit                      object
Value                    float64
Flag                      object
dtype: object

In [4]:
unwanted_columns = [ 
    "Flag","Year Code","Element Code", "Item Code"
]

df_clean_step_1 = df.drop(columns=unwanted_columns)
df_clean_step_1.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
0,2,Afghanistan,4,Algeria,"Cashew nuts, shelled",Export Quantity,2016,tonnes,3.0
1,2,Afghanistan,4,Algeria,"Cashew nuts, shelled",Export Value,2016,1000 US$,23.0
2,2,Afghanistan,4,Algeria,Crude materials,Export Value,2015,1000 US$,1.0
3,2,Afghanistan,4,Algeria,Crude materials,Export Value,2016,1000 US$,1.0
4,2,Afghanistan,4,Algeria,Crude materials,Export Value,2017,1000 US$,5.0


In [11]:
check_list = df_clean_step_1['Item'].value_counts()
check_list

Food prep nes               762746
Crude materials             642780
Pastry                      530694
Sugar confectionery         509540
Fruit, prepared nes         494784
                             ...  
Ghee, buffalo milk               4
Sugar flavoured                  4
Hides, camel, wet salted         2
Cake, hempseed                   2
Offals, other camelids           2
Name: Item, Length: 426, dtype: int64

In [12]:
no_support_data = [
    'Crude materials',
    'Cigarettes',
    'Pastry',
    'Food prep nes',
    'Fruit, prepared nes',
    'Beverages, non alcoholic',
    'Beverages, distilled alcoholic',
    'Beverages, fermented rice',
    'Beer of barley',
    'Beer of sorghum',
    'Bread', 
    'Infant food',
    'Wine',
    'Waters,ice etc',
    'Vitamins',
    'Wafers',
    'Oil, boiled etc',
    'Mixes and doughs',
    'Food Wastes',
    'Food preparations, flour, malt extract',
    'Cigars, cheroots',
    'Chocolate products nes']

df_clean_step_2 = df_clean_step_1[df_clean_step_1.Item.isin(no_support_data) == False]
df_clean_step_2.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
0,2,Afghanistan,4,Algeria,"Cashew nuts, shelled",Export Quantity,2016,tonnes,3.0
1,2,Afghanistan,4,Algeria,"Cashew nuts, shelled",Export Value,2016,1000 US$,23.0
5,2,Afghanistan,4,Algeria,Raisins,Export Quantity,2014,tonnes,12.0
6,2,Afghanistan,4,Algeria,Raisins,Export Value,2014,1000 US$,27.0
7,2,Afghanistan,4,Algeria,Spices nes,Export Quantity,2014,tonnes,0.0


In [7]:
import_data = ['Import Value']
df_import_value = df_clean_step_2[df_clean_step_2.Element.isin(import_data)]
df_import_value.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
12,2,Afghanistan,9,Argentina,Maize,Import Value,2017,1000 US$,68.0
19,2,Afghanistan,9,Argentina,"Meat, chicken",Import Value,2011,1000 US$,281.0
20,2,Afghanistan,9,Argentina,"Meat, chicken",Import Value,2014,1000 US$,1163.0
21,2,Afghanistan,9,Argentina,"Meat, chicken",Import Value,2015,1000 US$,159.0
22,2,Afghanistan,9,Argentina,"Meat, chicken",Import Value,2016,1000 US$,1653.0


In [8]:
df_export_value = df_clean_step_2.loc[df_clean_step_2['Element'] == 'Export Value']
df_export_value.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
1,2,Afghanistan,4,Algeria,"Cashew nuts, shelled",Export Value,2016,1000 US$,23.0
6,2,Afghanistan,4,Algeria,Raisins,Export Value,2014,1000 US$,27.0
8,2,Afghanistan,4,Algeria,Spices nes,Export Value,2014,1000 US$,0.0
34,2,Afghanistan,1,Armenia,Plums dried (prunes),Export Value,2019,1000 US$,0.0
38,2,Afghanistan,1,Armenia,Raisins,Export Value,2017,1000 US$,1.0


In [9]:
df_aus = df_clean_step_2.loc[df_clean_step_2['Reporter Countries'] == 'Australia']
df_aus = df_aus[df_aus['Unit'] != 'tonnes']
df_aus_import = df_aus[df_aus['Element'] == 'Import Value']
df_aus_import

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
627701,10,Australia,2,Afghanistan,Almonds shelled,Import Value,2012,1000 US$,11.0
627702,10,Australia,2,Afghanistan,Almonds shelled,Import Value,2013,1000 US$,11.0
627703,10,Australia,2,Afghanistan,Almonds shelled,Import Value,2014,1000 US$,7.0
627704,10,Australia,2,Afghanistan,Almonds shelled,Import Value,2017,1000 US$,9.0
627705,10,Australia,2,Afghanistan,Almonds shelled,Import Value,2018,1000 US$,5.0
...,...,...,...,...,...,...,...,...,...
1233483,10,Australia,181,Zimbabwe,"Vegetables, frozen",Import Value,1999,1000 US$,83.0
1233488,10,Australia,181,Zimbabwe,"Vegetables, preserved nes",Import Value,1997,1000 US$,0.0
1233489,10,Australia,181,Zimbabwe,"Vegetables, preserved nes",Import Value,2005,1000 US$,8.0
1233490,10,Australia,181,Zimbabwe,"Vegetables, preserved nes",Import Value,2006,1000 US$,26.0


In [10]:
df_aus_export = df_aus[df_aus['Element'] == 'Export Value']
df_aus_export


,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
627708,10,Australia,2,Afghanistan,Almonds shelled,Export Value,2018,1000 US$,2997.0
627722,10,Australia,2,Afghanistan,Apples,Export Value,1992,1000 US$,3.0
627742,10,Australia,2,Afghanistan,Asses,Export Value,1999,1000 US$,1.0
627744,10,Australia,2,Afghanistan,Bacon and ham,Export Value,2010,1000 US$,1.0
627754,10,Australia,2,Afghanistan,"Butter, cow milk",Export Value,1996,1000 US$,48.0
...,...,...,...,...,...,...,...,...,...
1233509,10,Australia,181,Zimbabwe,Wheat,Export Value,1996,1000 US$,0.0
1233510,10,Australia,181,Zimbabwe,Wheat,Export Value,1997,1000 US$,0.0
1233511,10,Australia,181,Zimbabwe,Wheat,Export Value,1998,1000 US$,0.0
1233512,10,Australia,181,Zimbabwe,Wheat,Export Value,2012,1000 US$,6262.0


In [20]:
df_aus_export[df_aus_export['Year'] == 2013]


,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item,Element,Year,Unit,Value
627901,10,Australia,2,Afghanistan,"Meat, cattle, boneless (beef & veal)",Export Value,2013,1000 US$,836.0
627938,10,Australia,2,Afghanistan,"Meat, sheep",Export Value,2013,1000 US$,384.0
627988,10,Australia,2,Afghanistan,"Offals, edible, cattle",Export Value,2013,1000 US$,16.0
628222,10,Australia,3,Albania,"Fat nes, prepared",Export Value,2013,1000 US$,302.0
628304,10,Australia,3,Albania,"Meat, cattle, boneless (beef & veal)",Export Value,2013,1000 US$,99.0
...,...,...,...,...,...,...,...,...,...
1231889,10,Australia,249,Yemen,Tea,Export Value,2013,1000 US$,9.0
1231945,10,Australia,249,Yemen,Wheat,Export Value,2013,1000 US$,264967.0
1232575,10,Australia,251,Zambia,"Milk, whole dried",Export Value,2013,1000 US$,1242.0
1232732,10,Australia,181,Zimbabwe,"Beans, dry",Export Value,2013,1000 US$,2.0
